In [2]:
!pip install -r ../requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: '../requirements.txt'

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


## 2.1 BPE of unknown words
Try the BPE tokenizer from the tiktoken library on the unknown words “Akwirw ier” 
and print the individual token IDs. 

Then, call the decode function on each of the resulting integers in this list 
to reproduce the mapping shown in figure 2.11. 

Lastly, call the decode method on the token IDs to 
check whether it can reconstruct the original input, “Akwirw ier.”

In [3]:
!pwd

/teamspace/studios/this_studio


In [4]:
import tiktoken

In [7]:
s ="Akwirw ier."
tokenizer = tiktoken.get_encoding("gpt2")
tokens = tokenizer.encode(s)
tokens 

[33901, 86, 343, 86, 220, 959, 13]

In [8]:
tokenizer.decode(tokens)

'Akwirw ier.'

In [55]:
## Implementing the GPT DataLoader class
import torch
from torch.utils.data import Dataset, DataLoader
 
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        """
        Params:
            stride: the number of indexes to move between batches
            max_length: the window size
        """
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i: i+max_length]
            target_chunk = token_ids[i+1: i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]



In [24]:
!ls .

LLMs-from-scratch  examples  getting_started.ipynb


In [25]:
with open("LLMs-from-scratch/ch02/01_main-chapter-code/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [26]:
# print(raw_text)

In [27]:
tokenizer = tiktoken.get_encoding("gpt2")
encoded_text = tokenizer.encode(raw_text)

In [28]:
len(encoded_text)

5145

In [42]:
dataset = GPTDatasetV1(raw_text, tokenizer, max_length=2, stride=2)

In [46]:
dataset[0]


(tensor([33901,    86]), tensor([ 86, 343]))

In [45]:
dataset[1]

(tensor([343,  86]), tensor([ 86, 220]))

In [47]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True, num_workers=0):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader

In [49]:
vocab_size = 50257
output_dim = 256
context_length = 1024
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [56]:
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length, stride=max_length)


In [58]:
for batch in dataloader:
    x, y = batch
    #print(x, y)
    token_embeddings = token_embedding_layer(x)
    pos_embeddings = pos_embedding_layer(torch.arange(max_length))

    input_embeddings = token_embeddings + pos_embeddings
    print(input_embeddings)
    break


tensor([[[-1.7198e+00,  2.3616e+00, -2.3608e-01,  ..., -2.1460e+00,
           5.5452e-01, -1.2040e+00],
         [-1.4681e+00, -4.6416e-01, -1.3035e+00,  ...,  7.8037e-01,
          -1.0259e+00, -5.8477e-01],
         [ 2.9149e-01,  1.2875e-01,  5.5797e-02,  ..., -5.2512e-01,
          -1.4244e+00,  2.5446e+00],
         [-8.3724e-02,  1.2154e+00, -4.4828e-01,  ..., -9.9454e-01,
          -4.0514e-01, -6.3021e-01]],

        [[-8.7138e-01,  1.6718e+00,  1.3357e+00,  ...,  6.0381e-01,
           6.0504e-01, -1.5115e+00],
         [-2.1012e+00,  3.7795e-01, -8.5915e-01,  ...,  7.4455e-01,
           2.4568e+00, -1.8294e+00],
         [-2.0261e+00,  2.4717e+00, -4.1007e-02,  ...,  1.0370e+00,
          -1.3871e+00,  1.0243e+00],
         [-8.0761e-04,  2.0683e+00, -1.5610e+00,  ...,  7.7741e-02,
           7.4035e-01, -9.1926e-01]],

        [[-1.5308e+00,  1.6588e+00,  4.8230e-01,  ...,  1.4478e+00,
           6.5506e-01, -2.4078e-01],
         [ 3.7883e-01, -2.4422e-01,  5.2323e-01,  .

## Positional Embeddings

In [60]:
torch.arange(1,5)

tensor([1, 2, 3, 4])